In [1]:
import numpy as np

# Tha hanging chain problem

$$
\min f_{8}(x) \; , \; f_{8}(x)=\sum_{i=1}^{nb} l_i(x)*\frac{y_i+y_{i-1}}{2} \\
h_i(x)=l_i(x)^2-L_i^2 \; , \; i=1,...,nb \\
l_i(x) = \sqrt{(x_i-x_{i-1})^2+(y_i-y_{i-1})^2} \\
\min f(x)=\sum_{i=1}^{nb} L_i*\frac{y_i+y_{i-1}}{2} \; , \; s.t. h(x)=0
$$

## Definition de l'oracle

In [112]:
def split_half(a):
    half = len(a)//2
    return a[:half], a[half:]

def f (X):
    x, y = split_half(X)
    global L
    global nb
    global _y0,_ylast
    F = []
    for i in range(nj+1):
        if i==0:
            F.append(L[i]*(y[i]+_y0)/2)
        elif i == (nj):
            F.append(L[i]*(_ylast+y[i-1])/2)
        else:
            F.append(L[i]*(y[i]+y[i-1])/2)
    return sum(F)

def g (X):
    global L
    global nb
    Gx, Gy = split_half(X)
    Gx.fill(0)
    for i in range(1,nj):
        Gy[i] = L[i]
    G = np.concatenate((Gx, Gy), axis=None)
    return G

def h (X):
    global nb,L
    global _x0,_y0
    x, y = split_half(X)
    nodes = []
    for i in range(nb+1):
        if i==0:
            nodes.append([_x0,_y0])
        elif i==nb:
            nodes.append([_xlast,_ylast])
        else:
            nodes.append([x[i-1],y[-1]])
    nodes = np.array( nodes )
    li = np.array([np.linalg.norm(nodes[i] - nodes[i-1]) for i in range(1,nb+1)])
    H = li**2 - L
    return H 

def j (X):
    global nj,nb
    x, y = split_half(X)
    ghx = []
    for jnb in range (nb+1):
        ghx.append([])
        for inj in range (nj+1):
            if inj == 0:
                ghx.append([2*(x[inj]-_x0)])
            elif inj == nj:
                ghx.append([2*(_xlast-x[inj-1])])
            else:
                ghx.append([2*(x[inj]-x[inj-1])*(inj==jnb) -2*(x[inj]-x[inj-1])*(inj==(jnb-1))])
    
    ghy = []
    for jnb in range (nb+1):
        ghy.append([])
        for inj in range (nj+1):
            if inj == 0:
                ghy.append([2*(y[inj]-_y0)])
            elif inj == nj:
                ghy.append([2*(_ylast-y[inj-1])])
            else:
                ghy.append([2*(y[inj]-y[inj-1])*(inj==jnb) -2*(y[inj]-y[inj-1])*(inj==(jnb-1))])
            

    J = np.array([np.concatenate((np.array(ghx[i]), np.array(ghy[i])), axis=None) for i in range(0,nb)])
    return J



def phi( X, Lamda, C):
    H,J = oracle.hj(X, mode=1)
    F,G = oracle.fg(X, mode=1)
    L = F + np.dot(Lamda.T,H)+C/2*np.linalg.norm(H)**2
    return L

def gphi( X, Lamda, C):
    H,J = oracle.hj(X, mode=2)
    F,G = oracle.fg(X, mode=2)
    GL = np.concatenate((G+np.dot(Lamda.T,J).sum()), axis=None)
    return GL

def l( X, Lamda, C):
    H,J = oracle.hj(X, mode=1)
    F,G = oracle.fg(X, mode=1)
    L = F + np.dot(Lamda.T,H)+C/2*np.linalg.norm(H)**2
    return L

def gl( X, Lamda, C):
    H,J = oracle.hj(X, mode=2)
    F,G = oracle.fg(X, mode=2)
    GL = np.concatenate((G+np.dot(Lamda.T,J), H), axis=None)
    return GL

class oracle:
        
    def fg(X, mode):
        if mode==1:
            return f(X),None 
        elif mode==2:
            return f(X),g(X)
        elif mode==3:
            return None,g(X)
        else:
            print('Not on the list')

    def hj(X, mode):
        if mode==1:
            return h(X),None 
        elif mode==2:
            return h(X),j(X)
        elif mode==3:
            return None,j(X)
        else:
            print('Not on the list')

    def lgl(X,Lamda,C, mode):
        if mode==1:
            return l( X, Lamda, C),None 
        elif mode==2:
            return l(X, Lamda, C),gl( X, Lamda, C)
        elif mode==3:
            return None,gl(X, Lamda, C, oracle)
        else:
            print('Not on the list')
            
    def phig(X,Lamda,C, mode):
        if mode==1:
            return phi( X, Lamda, C),None 
        elif mode==2:
            return phi(X, Lamda, C),gphi( X, Lamda, C)
        elif mode==3:
            return None,gphi(X, Lamda, C, oracle)
        else:
            print('Not on the list')
    

def Armijo(X,d,Lamda,C,MaxItLineSearch,t0 = 100, theta=0.2, m=0.001):
    p=0
    t = [t0]
    while True:
        _phi, _gphi = oracle.phig(X,Lamda,C, mode=2)
        
        _phi1, _gphi1 = oracle.phig(X+t[-1]*d,Lamda,C, mode=1)
        
        if _phi1 <= _phi1 + m*t[-1]*np.dot(_gphi,d):
            return t[-1]
        else:
            t.append(theta*t[-1])
            if p<MaxItLineSearch:
                p+=1
            else:
                print(f'Max iterations \n\t direction: {d} \n\t step:{t[-1]}')
                raise ValueError('Max iteration')

            
def algorithme2(x0,Lamda,C, MaxIt=1000,tol = 10e-6, MaxItLineSearch=1000, step = 100):
    k=0
    X =  [x0] 
    while True:
        f, g = oracle.phig(X[-1],Lamda,C, mode=2)
        if np.linalg.norm(g)<tol:
            print (f'Converged on {X[-1]}')
            
            return X[-1], X
        else:
            d = -g
            try:
                step = Armijo(X[-1],d,Lamda,C, MaxItLineSearch=MaxItLineSearch, t0 = 100)
            except ValueError:
                print('Max armijo line search iterations')
                return None,X                                
            X.append(X[-1] + step*d)
            if k<MaxIt:
                k+=1
            else:
                print('Max iterations')
                return None,X
            
def algorithm(x0,lamda0,c0, tol = 10e-6, MaxIt=1000, stepC=0.5):
    k=0
    C = [c0]
    Lamda = [lamda0]
    Xk =  [x0] 
    Xkh = []
    while True:
        Xk.append(0)
        Xkh.append([])
        Xk[-1],Xkh[-1] = algorithme2(Xk[-2],Lamda[-1],C[-1])
        lagrange, gradient = oracle.lgl(Xk[-1],Lamda[-1],C[-1], mode = 2)
        if np.linalg.norm(gradient)<tol:
            print (f'Converged on {X[-1]}')
            return X[-1],X,Lamda[-1],Lamda,C[-1],C
        else:
            hk,jk = oracle.hj(X[-1], mode=1)
            Lamda.append(Lamda[-1]+C[-1]*hk)
            C = C+stepC
            if k<MaxIt:
                k+=1
            else:
                print('Max iterations')
                return None,X,Lamda,C

nb = 5
nj = nb - 1
L = np.ones(nb)
lamda = np.array([ [0.5] for i in range(nj)])
c=0.5
_x0=100
_xlast=100

_y0=100
_ylast=100

X = np.ones(2*nj)

X,L,C = algorithm(X,lamda,0.5)



ValueError: shapes (1,4) and (5,) not aligned: 4 (dim 1) != 5 (dim 0)

In [114]:
H

NameError: name 'H' is not defined